In [1]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from simtk.openmm import app
import mdtraj as md
import os
import pickle
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping


INFO:rdkit:Enabling RDKit 2021.09.11 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


# Protonate SA RBD N501Y

In [2]:
# Load PDBs
rbd_pdb = app.PDBFile("../../input/SA_rbd_N501Y.pdb")
ace2_pdb = app.PDBFile("../../input/SA_ace2.pdb")


In [3]:
# Load the ffxmls
ffxml = ['/home/zhangi/choderalab/openmmforcefields/amber/ffxml/protein.ff14SB.xml', "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml", "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/tip3p_standard.xml"]
ff = app.ForceField(*ffxml)

In [4]:
# Add hydrogens to RBD
modeller = app.Modeller(rbd_pdb.topology, rbd_pdb.positions)
modeller.loadHydrogenDefinitions("/home/zhangi/choderalab/openmmforcefields/amber/glycam/hydrogens_formatted.xml")
modeller.addHydrogens(ff)

[None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'CYX',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 

In [5]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("../../input/SA_rbd_N501Y_protonated.pdb", "w"), keepIds=True)

# Make htf with truncated octahedron

In [6]:
import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    def _solvate(self,
               topology,
               positions,
               water_model,
               phase,
               ionic_strength,
               box_dimensions=None):
        """
        Generate a solvated topology, positions, and system for a given input topology and positions.
        For generating the system, the forcefield files provided in the constructor will be used.
        Parameters
        ----------
        topology : app.Topology
            Topology of the system to solvate
        positions : [n, 3] ndarray of Quantity nm
            the positions of the unsolvated system
        forcefield : SystemGenerator.forcefield
            forcefield file of solvent to add
        water_model : str
            solvent model to use for solvation
        phase : str
            if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
        ionic_strength : float * unit.molar
            the total concentration of ions (both positive and negative) to add using Modeller.
            
        Returns
        -------
        solvated_topology : app.Topology
            Topology of the system with added waters
        solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
            Solvated positions
        solvated_system : openmm.System
            The parameterized system, containing a barostat if one was specified.
        """
        print("solvating with truncated octahedron")
        modeller = app.Modeller(topology, positions)

        geompadding = 0.9 * unit.nanometers
        maxSize = max(max((pos[i] for pos in positions))-min((pos[i] for pos in positions)) for i in range(3))
        vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
        boxVectors = [(maxSize+geompadding)*v for v in vectors]

        # Now we have to add missing atoms
        if phase != 'vacuum':
            _logger.info(f"solvating at {ionic_strength} using {water_model}")
            modeller.addSolvent(self.system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
        else:
            pass

        solvated_topology = modeller.getTopology()
        solvated_positions = modeller.getPositions()

        # Canonicalize the solvated positions: turn tuples into np.array
        solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
        solvated_system = self.system_generator.create_system(solvated_topology)

        return solvated_topology, solvated_positions, solvated_system

In [7]:
# Generate htf
solvent_delivery = PointMutationExecutor2("../../input/SA_rbd_N501Y_protonated.pdb",
                        '1', # First chain is the barstar one
                        '501',
                        'ASN',
                        ligand_input="../../input/SA_ace2_protonated.pdb",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p


solvating with truncated octahedron
solvating with truncated octahedron


INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {2638: 2638, 2639: 2639, 2640: 2640, 2641: 2641, 2642: 2642, 2643: 2643, 2644: 2644}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB')]
INFO:setup:Changed particle 52's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 52's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 224's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 224's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 289's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 289's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 326's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 326's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 474's epsilon from 0.0 kJ/mol to 0.0001 kJ/mo

INFO:setup:Changed particle 980's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 980's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 1031's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 1031's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 1064's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 1064's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 1315's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 1315's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 1406's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 1406's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 1441's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 1441's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 1554's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 1554's sigma from 1.0 nm to 0.1 nm
INFO:setup:Changed particle 1

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


...

INFO:geometry:total reduced potential before atom placement: 19205.094284810082


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 0.27523557080441413
INFO:geometry:final reduced energy 19208.31316975746
INFO:geometry:sum of energies: 19205.369520380886
INFO:geometry:magnitude of difference in the energies: 2.943649376574269
INFO:geometry:Final logp_proposal: -2378.7753393100897


added energy components: [('CustomBondForce', 0.2870428687168419), ('CustomAngleForce', 2.387258774067527), ('CustomTorsionForce', 5.16550575820673), ('CustomBondForce', -7.564571830186683)]


INFO:setup:charge diff: 0
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=10.632500000000002, y=0

In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/32/1/0/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

In [9]:
traj = md.Trajectory(apo_htf.old_positions(apo_htf.hybrid_positions), md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "apo_old.pdb"))

traj = md.Trajectory(apo_htf.new_positions(apo_htf.hybrid_positions), md.Topology.from_openmm(apo_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "apo_new.pdb"))

In [13]:
for atom in apo_htf._topology_proposal.old_topology.atoms():
    if atom.index == 3001:
        print(atom.index, atom)

3001 <Atom 3001 (HG1) of chain 0 residue 193 (THR)>


In [15]:
for atom in apo_htf._topology_proposal.new_topology.atoms():
    if atom.index == 2994:
        print(atom.index, atom)

2994 <Atom 2994 (HG1) of chain 0 residue 193 (THR)>
